In [ ]:
import logging
import socket
import json
import os
import time
from eth_account.messages import encode_defunct
from web3 import Web3
from datetime import datetime
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad

# Set working directory
project_path = "D:/blockchain project"
if os.path.exists(project_path):
    os.chdir(project_path)
else:
    print(f"❌ Project path not found: {project_path}")
    exit(1)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Connect to Ganache
ganache_url = "http://127.0.0.1:8545"
web3 = Web3(Web3.HTTPProvider(ganache_url))
if not web3.is_connected():
    logger.error("❌ Failed to connect to Ganache. Ensure it's running.")
    exit(1)

# Load smart contract
try:
    with open("build/contracts/AuthSignatureManager.json", "r") as f:
        contract_json = json.load(f)
        abi = contract_json.get("abi")
        if not abi:
            raise KeyError("ABI is missing in JSON file")
except (FileNotFoundError, KeyError) as e:
    logger.error(f"❌ Smart contract error: {e}")
    exit(1)

contract_address = "0xD8B774A2273859AE7f7470cfe26cBCea985389DD"
contract = web3.eth.contract(address=contract_address, abi=abi)

from_address = web3.eth.accounts[0]
private_key = "0x98bd32f2046424bfe47139c137c1f6de9bf2297112961edaccd88e206190fe69"
logger.info(f"Using account: {from_address}")

# Verify private key matches from_address
account = web3.eth.account.from_key(private_key)
if account.address != from_address:
    logger.error(f"❌ Private key does not match address! Expected: {from_address}, Got: {account.address}")
    exit(1)

# Socket Setup
HOST = "127.0.0.1"
PORT = 55555

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

AUTHORIZED_CLIENTS = {}

ENCRYPTION_KEY = get_random_bytes(16)  # 16 bytes for AES-128
logger.info(f"🔑 Generated encryption key: {ENCRYPTION_KEY.hex()}")

def encrypt_message(message):
    cipher = AES.new(ENCRYPTION_KEY, AES.MODE_CBC)
    iv = cipher.iv
    padded_message = pad(message.encode(), AES.block_size)
    encrypted_message = cipher.encrypt(padded_message)
    return iv + encrypted_message

def get_message_hash(sender, message, context=""):
    start_time = time.time()
    sender = Web3.to_checksum_address(sender)
    hash_result = Web3.solidity_keccak(["address", "string"], [sender, message])
    end_time = time.time()
    logger.info(f"⏱ Hash generation time ({context}): {(end_time - start_time)*1000:.2f} ms")
    return hash_result

def verify_signature(message, signature, sender):
    start_time = time.time()
    message_hash = get_message_hash(sender, message, "for verification")
    prefixed_message = encode_defunct(hexstr=message_hash.hex())
    recovered = web3.eth.account.recover_message(prefixed_message, signature=signature)
    end_time = time.time()
    logger.info(f"⏱ Signature verification time: {(end_time - start_time)*1000:.2f} ms")
    return recovered == sender

def is_client_authorized(client_addr):
    if client_addr not in AUTHORIZED_CLIENTS:
        return False
    start_time = time.time()
    auth_status = contract.functions.authorizedClients(from_address).call()
    current_time = web3.eth.get_block('latest')['timestamp']
    is_expired = current_time >= auth_status[1]
    end_time = time.time()
    logger.info(f"⏱ Authorization check time: {(end_time - start_time)*1000:.2f} ms")
    return not is_expired

def authorize_client(conn, client_addr_str, message):
    logger.info(f"Attempting to authorize with message: {message}")
    
    auth_status = contract.functions.authorizedClients(from_address).call()
    current_time = web3.eth.get_block('latest')['timestamp']
    logger.info(f"Current auth status: {auth_status}, Current block time: {current_time}")
    
    if auth_status[2] and current_time < auth_status[1]:
        logger.info(f"✅ Client {client_addr_str} already authorized until {datetime.fromtimestamp(auth_status[1])}")
        AUTHORIZED_CLIENTS[client_addr_str] = auth_status[1]
        response_data = b"AUTHORIZED:" + ENCRYPTION_KEY  # Send key with response
        conn.sendall(response_data)
        return True

    start_time = time.time()
    message_hash = get_message_hash(from_address, message, "for authorization signature")
    prefixed_message = encode_defunct(hexstr=message_hash.hex())
    signed_message = web3.eth.account.sign_message(prefixed_message, private_key=private_key)
    signature = signed_message.signature
    sig_time = time.time()
    logger.info(f"⏱ Signature generation time: {(sig_time - start_time)*1000:.2f} ms")
    logger.info(f"Generated signature: {signature.hex()}")

    recovered = web3.eth.account.recover_message(prefixed_message, signature=signature)
    if recovered != from_address:
        error_msg = f"❌ Local signature verification failed. Expected: {from_address}, Recovered: {recovered}"
        logger.error(error_msg)
        conn.sendall(error_msg.encode())
        return False
    logger.info(f"✅ Local signature verification passed")

    try:
        gas_estimate = contract.functions.authorizeClient(signature, message).estimate_gas({'from': from_address})
        tx_start = time.time()
        tx = contract.functions.authorizeClient(signature, message).build_transaction({
            'from': from_address,
            'gas': gas_estimate + 50000,
            'nonce': web3.eth.get_transaction_count(from_address, 'pending'),
            'gasPrice': web3.eth.gas_price
        })
        signed_tx = web3.eth.account.sign_transaction(tx, private_key)
        tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)
        receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
        tx_end = time.time()
        logger.info(f"⏱ Transaction processing time: {(tx_end - tx_start)*1000:.2f} ms")

        if receipt.status == 1:
            expiry = contract.functions.authorizedClients(from_address).call()[1]
            AUTHORIZED_CLIENTS[client_addr_str] = expiry
            response_data = b"AUTHORIZED:" + ENCRYPTION_KEY  # Send key with response
            conn.sendall(response_data)
            logger.info(f"✅ Client {client_addr_str} authorized until {datetime.fromtimestamp(expiry)}")
            return True
        else:
            conn.sendall(b"ERROR: Transaction failed")
            logger.error(f"❌ Transaction failed with receipt: {receipt}")
            return False
    except ValueError as e:
        error_msg = f"❌ Authorization reverted: {str(e)}"
        logger.error(error_msg)
        conn.sendall(error_msg.encode())
        return False
    except Exception as e:
        error_msg = f"❌ Authorization error: {str(e)}"
        logger.error(error_msg)
        conn.sendall(error_msg.encode())
        return False


def handle_client(conn, addr):
    logger.info(f"🔗 New Connection: {addr}")
    client_authorized = False
    client_addr_str = f"{addr[0]}:{addr[1]}"

    try:
        while True:
            data = conn.recv(1024)
            if not data:
                break

            message = data.decode().strip()

            if not client_authorized:
                client_authorized = authorize_client(conn, client_addr_str, message)
                if not client_authorized:
                    break
                continue

            if not is_client_authorized(client_addr_str):
                logger.info(f"⏰ Authorization expired for {client_addr_str}")
                conn.sendall(b"REAUTH: Do you want to reauthorize? (yes/no)")
                response = conn.recv(1024).decode().strip().lower()
                
                if response == "yes":
                    client_authorized = authorize_client(conn, client_addr_str, "Authorize me")
                    if not client_authorized:
                        break
                    continue
                else:
                    logger.info(f"❌ Client {client_addr_str} chose not to reauthorize")
                    conn.sendall(b"DISCONNECT: Authorization expired")
                    break

            start_time = time.time()
            message_hash = get_message_hash(from_address, message, "for message signature")
            signed_message = web3.eth.account.sign_message(
                encode_defunct(hexstr=message_hash.hex()), 
                private_key=private_key
            )
            signature = signed_message.signature
            sig_time = time.time()
            logger.info(f"⏱ Signature generation time: {(sig_time - start_time)*1000:.2f} ms")

            if not verify_signature(message, signature, from_address):
                conn.sendall(b"ERROR: Signature verification failed")
                logger.error("❌ Signature verification failed locally")
                continue

            tx_start = time.time()
            try:
                gas_estimate = contract.functions.storeSignature(message, signature).estimate_gas({'from': from_address})
                tx = contract.functions.storeSignature(message, signature).build_transaction({
                    'from': from_address,
                    'gas': gas_estimate + 50000,
                    'nonce': web3.eth.get_transaction_count(from_address, 'pending'),
                    'gasPrice': web3.eth.gas_price
                })
                signed_tx = web3.eth.account.sign_transaction(tx, private_key)
                tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)
                receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
                tx_end = time.time()
                logger.info(f"⏱ Transaction processing time: {(tx_end - tx_start)*1000:.2f} ms")

                if receipt.status == 1:
                    encrypted_message = encrypt_message(message)
                    logger.info(f"🔒 Encrypted message: {encrypted_message.hex()}")
                    response_data = len(signature).to_bytes(4, 'big') + signature + encrypted_message
                    conn.sendall(response_data)
                    logger.info("✅ Message processed and stored")
                else:
                    logger.error(f"❌ Transaction failed with receipt: {receipt}")
                    conn.sendall(b"ERROR: Transaction failed")
            except ValueError as e:
                logger.error(f"❌ storeSignature reverted: {str(e)}")
                conn.sendall(f"ERROR: storeSignature reverted: {str(e)}".encode())
            except Exception as e:
                logger.error(f"❌ Transaction error: {e}")
                conn.sendall(f"ERROR: {str(e)}".encode())

    except Exception as e:
        logger.error(f"❌ Connection error: {e}")
    finally:
        conn.close()
        if client_addr_str in AUTHORIZED_CLIENTS:
            del AUTHORIZED_CLIENTS[client_addr_str]
        logger.info(f"❌ Client {client_addr_str} disconnected")


try:
    start_time = time.time()
    server_socket.bind((HOST, PORT))
    server_socket.listen(5)
    end_time = time.time()
    logger.info(f"⏱ Server startup time: {(end_time - start_time)*1000:.2f} ms")
    logger.info(f"🟢 Server is running on {HOST}:{PORT}")

    while True:
        conn, addr = server_socket.accept()
        handle_client(conn, addr)
except KeyboardInterrupt:
    logger.info("🛑 Server shutting down.")
finally:
    server_socket.close()
    logger.info("🔌 Server socket closed.")


2025-03-10 18:27:51,468 - INFO - Using account: 0x4797808D0CC0b29D6acF3BC6cCF61292aE2123d5
2025-03-10 18:27:51,476 - INFO - 🔑 Generated encryption key: 2c960e16a4bddb4c2596bdb3db3042ee
2025-03-10 18:27:51,482 - INFO - ⏱ Server startup time: 0.13 ms
2025-03-10 18:27:51,484 - INFO - 🟢 Server is running on 127.0.0.1:55555
2025-03-10 18:27:57,109 - INFO - 🔗 New Connection: ('127.0.0.1', 52462)
2025-03-10 18:27:57,111 - INFO - Attempting to authorize with message: Authorize me
2025-03-10 18:27:57,141 - INFO - Current auth status: ['0x0000000000000000000000000000000000000000', 0, False], Current block time: 1741611404
2025-03-10 18:27:57,153 - INFO - ⏱ Hash generation time (for authorization signature): 9.54 ms
2025-03-10 18:27:57,166 - INFO - ⏱ Signature generation time: 22.24 ms
2025-03-10 18:27:57,168 - INFO - Generated signature: 2feb1c4b0bfbdc722fae1bb13f79a6bf2debfe7d4545345c727dfe217251637c00a798e4e6be1fde0a269ede2b1a6f1ef0603d527fe174b63b27264f87868a4f1b
2025-03-10 18:27:57,181 - INF